# 图像部分提取特征

In [ ]:
%%bash
cp '/content/drive/MyDrive/SIFT.zip' -r /content/
unzip SIFT.zip
mv dataset/2021MCM_ProblemC_Files_filtered dataset/2021MCM_ProblemC_Files

In [12]:
mv dataset/2021MCM_ProblemC_Files_filtered dataset/2021MCM_ProblemC_Files

In [ ]:
%%bash
python preprocess.py
python main.py |tee output.txt

In [ ]:
%%bash
pip install imgaug==0.2.6
pip install opencv-python==3.4.2.16
pip install opencv-contrib-python==3.4.2.16

In [23]:
import pickle
from random import choice
from sklearn.cluster import KMeans
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

sc = StandardScaler()

def des2feature(centers, des, num_words):
    img_feature_vec = np.zeros([num_words])
    for fi in des:
        diffMat = np.tile(fi, (15, 1)) - centers
        sqSum = (diffMat**2).sum(axis=1)
        dist = sqSum**0.5
        sortedIndices = dist.argsort()
        idx = sortedIndices[0] # index of the nearest center
        img_feature_vec[idx] += 1
    return img_feature_vec

def get_all_features(dess, num_words):
    allvec = np.zeros((len(dess), num_words),'float32')
    for i in range(len(dess)):
        if dess[i].any() != None:
            allvec[i] = des2feature(centers=centers, des=dess[i], num_words=num_words)
    return allvec

df = open('input_x.pkl','rb')
input_x = pickle.load(df)
df.close()

df = open('features_train.pkl', 'rb')
dess = pickle.load(df)
df.close()

df = open('labels_train.pkl', 'rb')
train_labels = pickle.load(df)
df.close()

df = open("globalid.pkl", 'rb')
globalids = pickle.load(df)
df.close()

kmeans = KMeans(n_clusters=15, random_state=41).fit(input_x)
centers = kmeans.cluster_centers_

output = open('centers.pkl', 'wb')
pickle.dump(centers, output)
output.close()

allvec = get_all_features(dess, 15)

X_train = sc.fit_transform(allvec)

In [24]:
print(len(X_train))
print(len(globalids))

1621
1621


In [25]:
# 计算两种均值，一个是positve时候X_train avg
# output X_trainPositiveMean, X_trainNegativeMean

Positivedata = []
Negativedata = []
positive = 0
negative = 0
for f, label in zip(X_train, train_labels):
    if label == 1:
        Positivedata.append(f)
        positive += 1
    if label == 0:
        Negativedata.append(f)
        negative += 1


X_trainPositiveMean = np.sum(Positivedata, axis=0)/positive
X_trainNegativeMean = np.sum(Negativedata, axis=0)/negative

In [94]:
result222 = (X_trainPositiveMean *14 + X_trainNegativeMean * 2096)/(4 + 2096)


In [95]:
print(result222)

[ 6.4473206e-05 -5.9829350e-04 -5.0758454e-04 -3.6152362e-04
  9.0546157e-05 -3.5537084e-04 -4.1671458e-04 -2.2636596e-04
  4.6055657e-06  2.9804252e-05 -2.9390835e-04 -1.7931030e-04
 -2.3572627e-04  6.4345935e-05  1.5062344e-04]


In [26]:
print(type(X_trainPositiveMean))
print(X_trainNegativeMean)

<class 'numpy.ndarray'>
[ 6.1874057e-04 -5.7414691e-03 -4.8713032e-03 -3.4691719e-03
  8.6882955e-04 -3.4100371e-03 -3.9983345e-03 -2.1722291e-03
  4.4383123e-05  2.8569065e-04 -2.8202103e-03 -1.7212136e-03
 -2.2620209e-03  6.1726948e-04  1.4454441e-03]


# 模型部分
- 最后可以显示recall, accuracy, preicision, f1

In [27]:
import random
from math import radians, cos, sin, asin, sqrt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from datetime import datetime

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('averaged_perceptron_tagger')

In [29]:
df = pd.read_csv("/content/dataset/cleaned_data_VM.csv", encoding="utf-8")
col_n = ["Notes", "Lab Status", "Latitude", "Longitude", "Detection Date", "GlobalID"]
df = pd.DataFrame(df, columns = col_n)
wnl = WordNetLemmatizer()
data = df[df["Lab Status"].isin(["Positive ID", "Negative ID"])]

In [ ]:
print(df.info())

In [31]:
positive = ['underground', 'burrow', 'trunk', 'spring', 'summer', 'fall', 'yellow', 'black', 'spot', 'wood', 'huge', 'big', 'large', 'forest']
negative = ['red', 'wall', 'building', 'lawn', 'small', 'house', 'white', 'limb', 'eave', 'roof', 'apartment', 'window', 'dirt', 'possible', 'winter']
special = ['WSDA']

In [32]:
p_sum = [0 for n in positive]
n_sum = [0 for n in negative]

cluster_center = [(48.777534, -122.418612), (49.149394, -123.943134), (48.9833003333333, -122.577144), (48.9756055, -122.68818075), (49.025831, -122.6995478), (49.060215, -122.641648)]
def encoder(lemmatized):
  p = []
  n = []
  for word in positive:
    if word in lemmatized:
      p.append(1)
      p_sum[positive.index(word)] += 1
    else:
      p.append(0)
  for word in negative:
    if word in lemmatized:
      n.append(1)
      n_sum[negative.index(word)] += 1
    else:
      n.append(0)
  return p, n

def geo_distance(lon1, lat1, lon2, lat2):  # 地理距离
  """
  Calculate the great circle distance between two points
  on the earth (specified in decimal degrees)
  """
  lon1, lat1, lon2, lat2 = map(radians, map(float, [lon1, lat1, lon2, lat2]))
  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
  c = 2 * asin(sqrt(a))
  r = 6371
  return c * r

def get_min_distance(lon, lat):
  min = 999999999999
  for (lat_c, lon_c) in cluster_center:
    distance = geo_distance(lon, lat, lon_c, lat_c)
    if min > distance:
      min = distance
  return min

In [ ]:
print(data.info())

In [34]:
# get the date array
# accuracte to month
def get_date(date):
  match = re.search(r'\d{4}-\d{1,2}-\d{1,2}', str(date))
  if match == None:
    print(date)
    print("why this is wrong")
  tmp = datetime.strptime(match.group(), '%Y-%m-%d').date()
  month = tmp.month
  year = tmp.year
  return [year, month]

In [35]:
get_date('2000-02-05')

[2000, 2]

In [36]:
def load_dataset():
  train = []
  train_label = []
  positive = 0
  positive_features = []
  negative = 0
  for index, row_data in data.iterrows():
    note = row_data["Notes"]
    lat = row_data["Latitude"]
    lon = row_data["Longitude"]
    # lat always around 49, Lon always around 100+
    min_distance = [get_min_distance(lon, lat)]

    # if row_data["Lab Status"] == "Positive ID":
    #   print(min_distance)
    timearray = [*get_date(row_data["Detection Date"])]

    # deal with picture
    globalid = row_data["GlobalID"]
    if globalid in globalids:
      indexs = [index for (index,value) in enumerate(globalids) if value == globalid]
      # 取最大值X_train
      maxSum = -99999999
      for i in range(len(indexs)):
        if np.sum(X_train[indexs[i]]) > maxSum:
          maxSum = np.sum(X_train[indexs[i]])
      if maxSum == -99999999:
        print("Something went wrong check maxSum")
      picturearray = X_train[indexs[i]].tolist()
    elif row_data["Lab Status"]=="Positive ID":
      picturearray = X_trainPositiveMean.tolist()
    elif row_data["Lab Status"]=="Negative ID":
      picturearray = X_trainNegativeMean.tolist()
    else:
      # other status retain unprocessed
      pass
    

    s = []
    for sp in special:
      if sp in note:
        s.append(1)
      else:
        s.append(0)
    note = note.lower()
    tokenized_note = nltk.word_tokenize(note)
    tagged = nltk.pos_tag(tokenized_note)
    lemmatized = []
    for word, tag in tagged:
      if tag.startswith('NN'):
        lemmatized.append(wnl.lemmatize(word, pos='n'))
      elif tag.startswith('JJ'):
        lemmatized.append(wnl.lemmatize(word, pos='a'))
    p, n = encoder(lemmatized)

    if row_data["Lab Status"]=="Positive ID":
      label = 1
      positive += 1
      positive_features.append(np.asarray(p+n+s+min_distance+timearray+picturearray, dtype=np.double))
    elif row_data["Lab Status"]=="Negative ID":
      label = 0
      negative += 1
    elif row_data["Lab Status"]=="Unprocessed":
      continue
    else:
      continue
    train.append(np.asarray(p+n+s+min_distance+timearray+picturearray, dtype=np.double))
    train_label.append(np.asarray(label, dtype=np.int))
  while positive < negative / 10:
    r = random.random()
    if r < 0.5:
      r_special = [1]
    else:
      r_special = [0]
    enhance = random.choice(positive_features)
    train.append(np.asarray(enhance, dtype=np.int))
    train_label.append(np.asarray(1, dtype=np.int))
    positive += 1
  assert len(train) == len(train_label)
  return train, train_label

In [37]:
from sklearn.model_selection import cross_val_score #交叉验证模块
from sklearn.linear_model import LogisticRegression #逻辑回归
from sklearn.tree import DecisionTreeClassifier #决策树
from sklearn.svm import SVC #支持向量机
from sklearn.ensemble import RandomForestClassifier #随机森林
from sklearn.datasets.base import Bunch
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score
from sklearn.utils import resample

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
train, train_label = load_dataset()
accuracy_scores = cross_val_score(RandomForestClassifier(n_estimators=10), train, train_label, cv=10, scoring='accuracy')
print(np.average(accuracy_scores))
precision_scores = cross_val_score(RandomForestClassifier(n_estimators=10), train, train_label, cv=10, scoring='precision')
print(np.average(precision_scores))
recall_scores = cross_val_score(RandomForestClassifier(n_estimators=10), train, train_label, cv=10, scoring='recall')
print(np.average(recall_scores))
F1_scores = cross_val_score(RandomForestClassifier(n_estimators=10), train, train_label, cv=10, scoring='f1')
print(np.average(F1_scores))

# validation
- 使用新的unprocessed的数据来validate我们的模型是否正常，由于过拟合

## baked version
- for split and validate

In [39]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

In [40]:
# with rename and split
def load_dataset2():
  train = []
  train_label = []
  positive = 0
  positive_features = []
  negative = 0
  for index, row_data in data.iterrows():
    note = row_data["Notes"]
    lat = row_data["Latitude"]
    lon = row_data["Longitude"]
    # lat always around 49, Lon always around 100+
    min_distance = [get_min_distance(lon, lat)]
    # if row_data["Lab Status"] == "Positive ID":
    #   print(min_distance)
    timearray = [*get_date(row_data["Detection Date"])]
    print(type(timearray))


    # deal with picture
    globalid = row_data["GlobalID"]
    if globalid in globalids:
      indexs = [index for (index,value) in enumerate(globalids) if value == globalid]
      # 取最大值X_train
      maxSum = -99999999
      for i in range(len(indexs)):
        if np.sum(X_train[indexs[i]]) > maxSum:
          maxSum = np.sum(X_train[indexs[i]])
      if maxSum == -99999999:
        print("Something went wrong check maxSum")
      picturearray = X_train[indexs[i]].tolist()
    elif row_data["Lab Status"]=="Positive ID":
      picturearray = X_trainPositiveMean.tolist()
    elif row_data["Lab Status"]=="Negative ID":
      picturearray = X_trainNegativeMean.tolist()
    else:
      # other status retain unprocessed
      pass


    s = []
    for sp in special:
      if sp in note:
        s.append(1)
      else:
        s.append(0)
    note = note.lower()
    tokenized_note = nltk.word_tokenize(note)
    tagged = nltk.pos_tag(tokenized_note)
    lemmatized = []
    for word, tag in tagged:
      if tag.startswith('NN'):
        lemmatized.append(wnl.lemmatize(word, pos='n'))
      elif tag.startswith('JJ'):
        lemmatized.append(wnl.lemmatize(word, pos='a'))
    p, n = encoder(lemmatized)
    if row_data["Lab Status"]=="Positive ID":
      label = 1
      positive += 1
      positive_features.append(np.asarray(p+n+s+min_distance+timearray+picturearray, dtype=np.double))
    elif row_data["Lab Status"]=="Negative ID":
      label = 0
      negative += 1
    elif row_data["Lab Status"]=="Unprocessed":
      continue
    else:
      continue
    train.append(np.asarray(p+n+s+min_distance+timearray+picturearray, dtype=np.double))
    train_label.append(np.asarray(label, dtype=np.int))
  while positive < negative / 10:
    r = random.random()
    if r < 0.5:
      r_special = [1]
    else:
      r_special = [0]
    enhance = random.choice(positive_features)
    train.append(np.asarray(enhance, dtype=np.int))
    train_label.append(np.asarray(1, dtype=np.int))
    positive += 1
  assert len(train) == len(train_label)

  # split and rename part
  trainfinal, test, train_labelfinal, test_label = train_test_split(train, train_label, stratify = train_label, test_size = 0.3,random_state = 41)
  return trainfinal, train_labelfinal, test, test_label

## different types of evaluation
- see below

In [41]:
train, train_label = load_dataset()
accuracy_scores = cross_val_score(RandomForestClassifier(n_estimators=10), train, train_label, cv=10, scoring='accuracy')
print(np.average(accuracy_scores))
precision_scores = cross_val_score(RandomForestClassifier(n_estimators=10), train, train_label, cv=10, scoring='precision')
print(np.average(precision_scores))
recall_scores = cross_val_score(RandomForestClassifier(n_estimators=10), train, train_label, cv=10, scoring='recall')
print(np.average(recall_scores))
F1_scores = cross_val_score(RandomForestClassifier(n_estimators=10), train, train_label, cv=10, scoring='f1')
print(np.average(F1_scores))

0.9947368421052631
0.9952380952380953
0.9476190476190476
0.95


In [ ]:
train, train_label, test, test_label = load_dataset2()
rs = RandomForestClassifier(n_estimators=100, random_state=42, max_features = 'sqrt', n_jobs=-1, verbose = 1)
rs.fit(train, train_label)

In [44]:
n_nodes = []
max_depths = []

for ind_tree in rs.estimators_:
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

Average number of nodes 35
Average maximum depth 7


In [ ]:
train_rf_predictions = rs.predict(train)
train_rf_probs = rs.predict_proba(train)[:, 1]

rf_predictions = rs.predict(test)
rf_probs = rs.predict_proba(test)[:, 1]

In [46]:
def evaluate_model(predictions, probs, train_predictions, train_probs):
    """Compare machine learning model to baseline performance.
    Computes statistics and shows ROC curve."""
    
    baseline = {}
    
    baseline['recall'] = recall_score(test_label, [1 for _ in range(len(test_label))])
    baseline['precision'] = precision_score(test_label, [1 for _ in range(len(test_label))])
    baseline['roc'] = 0.5
    
    results = {}
    
    results['recall'] = recall_score(test_label, predictions)
    results['precision'] = precision_score(test_label, predictions)
    results['roc'] = roc_auc_score(test_label, probs)
    
    train_results = {}
    train_results['recall'] = recall_score(train_label, train_predictions)
    train_results['precision'] = precision_score(train_label, train_predictions)
    train_results['roc'] = roc_auc_score(train_label, train_probs)
    
    for metric in ['recall', 'precision', 'roc']:
        print(f'{metric.capitalize()} Baseline: {round(baseline[metric], 2)} Test: {round(results[metric], 2)} Train: {round(train_results[metric], 2)}')
    
    # Calculate false positive rates and true positive rates
    base_fpr, base_tpr, _ = roc_curve(test_label, [1 for _ in range(len(test_label))])
    model_fpr, model_tpr, _ = roc_curve(test_label, probs)

    plt.figure(figsize = (8, 6))
    plt.rcParams['font.size'] = 16
    
    # Plot both curves
    plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
    plt.plot(model_fpr, model_tpr, 'r', label = 'model')
    plt.legend();
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('ROC Curves');


In [ ]:
evaluate_model(rf_predictions, rf_probs, train_rf_predictions, train_rf_probs)

In [48]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Oranges):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    Source: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.figure(figsize = (10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size = 24)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size = 14)
    plt.yticks(tick_marks, classes, size = 14)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    # Labeling the plot
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), fontsize = 20,
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
        
    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True label', size = 18)
    plt.xlabel('Predicted label', size = 18)

In [ ]:
cm = confusion_matrix(test_label, rf_predictions)
plot_confusion_matrix(cm, classes = ['VM', 'not VM'],
                      title = 'VM_classification Confusion Matrix')

In [50]:
feature = []
for i in range(48):
  nametmp = "x"+str(i)
  feature.append(nametmp)

In [ ]:
fi = pd.DataFrame({'feature': feature,'importance': rs.feature_importances_}).sort_values('importance', ascending = False)
fi.head()

# 14  p
# 15  n  
# 1   s     (29)
# 1  min_distance （30）
# 1  timearray （31）

# 计算probability
- 解答第二题中 likelihood of a mistaken classification

In [52]:
print(len(test))

683


In [ ]:
rs.predict_proba(test)

In [54]:
len(test)

683

In [ ]:
t = rs.predict_proba(test).ravel()
with open("result.txt") as f:
  for value in t:
    f.write(value)
    f.write("\n")

In [62]:
np.set_printoptions(threshold=np.inf)
with open("result.txt") as f:
  for i in range(test):
    f.write(test[i][0])
    f.write("\t")
    f.write(test[i][1])
    f.write("\n")

TypeError: ignored

In [56]:
# f = open("testwrite.txt", "w")
# str1 = np.array_str(rs.predict_proba(test)) 
# f.write(str1)
# f.close()

In [ ]:
import os
dir = './dataset/2021MCM_ProblemC_Files/'
files = os.listdir(dir)
print(files)

['ATT250_20200526_145556_1.jpg', 'ATT3252_16027535615382330455348063878828_1.jpg', 'ATT1056_67E53F9A-2FEA-4EFF-9E8B-90B212D7C3C3_2.jpg', 'ATT2017_IMG_1539[1]_2.jpg', 'ATT2998_IMG_6902_1.jpg', 'ATT2158_IMG-7819_2.jpg', 'ATT2412_67F10EB3-7273-4B45-8DBB-F81E0AF0DE03_1.jpg', 'ATT946_E9F757B3-F767-4049-942B-98E43A300D2D_1.jpg', 'ATT1055_A009153D-E0EF-40A4-9630-9BD677BE268B_1.jpg', 'ATT943_D6A10AE4-9759-4B37-A406-5B470D3EB25C_1.jpg', 'ATT2109_20200818_131645_2.jpg', 'ATT3205_PXL_20201007_201043731_1.jpg', 'ATT1544_image_2.jpg', 'ATT1138_20200723_120706_1.jpg', 'ATT794_bee 6_1.jpg', 'ATT2938_Bee_1.jpg', 'ATT452_20200606_161003_1.jpg', 'ATT2877_IMG_20200911_083816[1]_1.jpg', 'ATT3303_5F5E645F-0249-4CF9-84E2-9E7BCD1CE348_1.jpg', 'ATT209_04E429B7-CB1E-49C1-A375-55E2B7332361_1.jpg', 'ATT2638_IMG_3820_2.jpg', 'ATT130_IMG_20200503_193640_177_2.jpg', 'ATT1285_image_1.jpg', 'ATT1066_34DCB50E-BE73-4936-93AB-85FD56DDFC36_1.jpg', 'ATT2149_IMG_3201_3.jpg', 'ATT2171_IMG_20200819_165341_1.jpg', 'ATT1667_IM

In [ ]:
%%bash
python preprocess.py

In [ ]:
%%bash
python main.py |tee output.txt

# 解决第五题
- 什么表明当前的蝗虫在华盛顿州被灭绝了

In [68]:
longitudeHighest = -116.95
longitudeLowest = -124.8
latitudeHighest = 49
latitudeLowest = 45.53333333

In [69]:
def forWangshington():
  data = pd.read_csv("/content/dataset/cleaned_data_VM.csv", encoding="utf-8")
  col_n = ["Notes", "Lab Status", "Latitude", "Longitude", "Detection Date", "GlobalID"]
  data = pd.DataFrame(data, columns = col_n)
  data2 = pd.DataFrame(col_n)
  for index, row in data.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    if lat > latitudeHighest:
      continue
    if lat < latitudeLowest:
      continue
    if lon > longitudeHighest:
      continue
    if lon < longitudeLowest:
      continue
    data2 = data2.append({'GlobalID':row["GlobalID"],'Detection Date':row["Detection Date"],'Notes':row["Notes"],'Lab Status':row["Lab Status"],'Latitude':row["Latitude"],'Longitude':row["Longitude"]},ignore_index=True)
  
  return data2


In [ ]:
dataWashingtonState.info()

In [71]:
dataWashingtonState = forWangshington()

In [72]:
dataWashingtonState = dataWashingtonState[dataWashingtonState["Lab Status"].isin(["Positive ID", "Negative ID", "Unverified"])]

In [84]:
dataWashingtonState.to_csv("result")

In [73]:
def get_date(date):
  match = re.search(r'\d{4}-\d{1,2}-\d{1,2}', str(date))
  if match == None:
    return [2020, 2]
  tmp = datetime.strptime(match.group(), '%Y-%m-%d').date()
  month = tmp.month
  year = tmp.year
  return [year, month]

In [74]:
# with rename and split
def load_test_dataset():
  test = []
  positive = 0
  positive_features = []
  negative = 0
  for index, row_data in dataWashingtonState.iterrows():
    note = row_data["Notes"]
    lat = row_data["Latitude"]
    lon = row_data["Longitude"]
    # lat always around 49, Lon always around 100+
    min_distance = [get_min_distance(lon, lat)]
    # if row_data["Lab Status"] == "Positive ID":
    #   print(min_distance)
    timearray = [*get_date(row_data["Detection Date"])]

    if timearray[1] <= 6:
      timearray[1] = timearray[1] +6
    else:
      timearray[0] = timearray[0] + 1
      timearray[1] = timearray[1] + 6 - 12

    # deal with picture
    globalid = row_data["GlobalID"]
    if globalid in globalids:
      indexs = [index for (index,value) in enumerate(globalids) if value == globalid]
      # 取最大值X_train
      maxSum = -99999999
      for i in range(len(indexs)):
        if np.sum(X_train[indexs[i]]) > maxSum:
          maxSum = np.sum(X_train[indexs[i]])
      if maxSum == -99999999:
        print("Something went wrong check maxSum")
      picturearray = X_train[indexs[i]].tolist()
    elif row_data["Lab Status"]=="Positive ID":
      picturearray = X_trainPositiveMean.tolist()
    elif row_data["Lab Status"]=="Negative ID":
      picturearray = X_trainNegativeMean.tolist()
    else:
      # other status retain unprocessed
      pass

    s = []
    for sp in special:
      if sp in note:
        s.append(1)
      else:
        s.append(0)
    note = note.lower()
    tokenized_note = nltk.word_tokenize(note)
    tagged = nltk.pos_tag(tokenized_note)
    lemmatized = []
    for word, tag in tagged:
      if tag.startswith('NN'):
        lemmatized.append(wnl.lemmatize(word, pos='n'))
      elif tag.startswith('JJ'):
        lemmatized.append(wnl.lemmatize(word, pos='a'))
    p, n = encoder(lemmatized)
    if row_data["Lab Status"]=="Positive ID":
      positive += 1
      positive_features.append(np.asarray(p+n+s+min_distance+timearray+picturearray, dtype=np.double))
    elif row_data["Lab Status"]=="Negative ID":
      negative += 1
    elif row_data["Lab Status"]=="Unprocessed":
      continue
    else:
      continue
    test.append(np.asarray(p+n+s+min_distance+timearray+picturearray, dtype=np.double))
  return test

In [ ]:
test = load_test_dataset()
np.set_printoptions(threshold=np.inf)
print(rs.predict_proba(test)) 
result = rs.predict_proba(test)

In [ ]:
print(test)

In [ ]:
print(result)

In [ ]:
negativecase = 0
positivecase = 0
positiveExtreme = 0
listfinal = []
for i in range(len(rs.predict_proba(test))):
  listtmp = rs.predict_proba(test)[i]
  if listtmp[0] > 0.8:
    negativecase += 1

  elif listtmp[1]> 0.8:
    positiveExtreme += 1
    listfinal.append(listtmp)
  elif listtmp[0] <= 0.8:
    positivecase += 1

print(negativecase)
print(positivecase)
print(positiveExtreme)

# 14
# 2

In [ ]:
print(listfinal)

# 解决第三题
- priorize investigation
- 处理unverified数据集

In [85]:
data = pd.read_csv("/content/dataset/cleaned_data_VM.csv", encoding="utf-8")
col_n = ["Notes", "Lab Status", "Latitude", "Longitude", "Detection Date", "GlobalID"]
data = pd.DataFrame(data, columns = col_n)

dataunverified = data[data["Lab Status"].isin(["Unverified"])]

In [ ]:
dataunverified.info()

In [100]:
def load_test_dataset():
  test = []
  positive = 0
  positive_features = []
  negative = 0
  for index, row_data in dataunverified.iterrows():
    note = row_data["Notes"]
    lat = row_data["Latitude"]
    lon = row_data["Longitude"]
    # lat always around 49, Lon always around 100+
    min_distance = [get_min_distance(lon, lat)]
    # if row_data["Lab Status"] == "Positive ID":
    #   print(min_distance)
    timearray = [*get_date(row_data["Detection Date"])]

    if timearray[1] <= 6:
      timearray[1] = timearray[1] +6
    else:
      timearray[0] = timearray[0] + 1
      timearray[1] = timearray[1] + 6 - 12

    # deal with picture
    globalid = row_data["GlobalID"]
    if globalid in globalids:
      indexs = [index for (index,value) in enumerate(globalids) if value == globalid]
      maxSum = -99999999
      for i in range(len(indexs)):
        if np.sum(X_train[indexs[i]]) > maxSum:
          maxSum = np.sum(X_train[indexs[i]])
      if maxSum == -99999999:
        print("Something went wrong check maxSum")
      picturearray = X_train[indexs[i]].tolist()
    elif row_data["Lab Status"]=="Positive ID":
      picturearray = X_trainPositiveMean.tolist()
    elif row_data["Lab Status"]=="Negative ID":
      picturearray = X_trainNegativeMean.tolist()
    else:
      picturearray = result222.tolist()

    s = []
    for sp in special:
      if sp in note:
        s.append(1)
      else:
        s.append(0)
    note = note.lower()
    tokenized_note = nltk.word_tokenize(note)
    tagged = nltk.pos_tag(tokenized_note)
    lemmatized = []
    for word, tag in tagged:
      if tag.startswith('NN'):
        lemmatized.append(wnl.lemmatize(word, pos='n'))
      elif tag.startswith('JJ'):
        lemmatized.append(wnl.lemmatize(word, pos='a'))
    p, n = encoder(lemmatized)
    print(len(test))
    test.append(np.asarray(p+n+s+min_distance+timearray+picturearray, dtype=np.double))
  return test

In [101]:
test = load_test_dataset()
np.set_printoptions(threshold=np.inf)
print(rs.predict_proba(test))
result = rs.predict_proba(test)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished


[[0.1  0.9 ]
 [0.18 0.82]
 [0.01 0.99]
 [0.45 0.55]
 [0.46 0.54]
 [0.45 0.55]
 [0.39 0.61]
 [0.46 0.54]
 [0.36 0.64]
 [0.46 0.54]
 [0.45 0.55]
 [0.44 0.56]
 [0.41 0.59]
 [0.49 0.51]
 [0.46 0.54]
 [0.46 0.54]
 [0.46 0.54]
 [0.49 0.51]
 [0.47 0.53]
 [0.43 0.57]
 [0.48 0.52]
 [0.48 0.52]
 [0.46 0.54]
 [0.47 0.53]
 [0.47 0.53]
 [0.47 0.53]
 [0.45 0.55]
 [0.37 0.63]
 [0.46 0.54]
 [0.46 0.54]
 [0.47 0.53]
 [0.38 0.62]
 [0.47 0.53]
 [0.45 0.55]
 [0.38 0.62]
 [0.46 0.54]
 [0.47 0.53]
 [0.46 0.54]
 [0.47 0.53]
 [0.46 0.54]
 [0.47 0.53]
 [0.43 0.57]
 [0.42 0.58]
 [0.49 0.51]
 [0.48 0.52]
 [0.45 0.55]
 [0.46 0.54]
 [0.47 0.53]
 [0.46 0.54]
 [0.36 0.64]
 [0.47 0.53]
 [0.5  0.5 ]
 [0.46 0.54]
 [0.41 0.59]
 [0.44 0.56]
 [0.45 0.55]
 [0.46 0.54]
 [0.49 0.51]
 [0.46 0.54]
 [0.47 0.53]
 [0.45 0.55]
 [0.46 0.54]
 [0.46 0.54]
 [0.48 0.52]
 [0.46 0.54]
 [0.46 0.54]
 [0.45 0.55]
 [0.1  0.9 ]
 [0.4  0.6 ]
 [0.46 0.54]
 [0.38 0.62]
 [0.48 0.52]
 [0.47 0.53]
 [0.46 0.54]
 [0.46 0.54]
 [0.48 0.52]
 [0.43 0.57]

In [102]:
test = load_test_dataset()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [107]:
negativecase = 0
positivecase = 0
positiveExtreme = 0
listfinal = []
for i in range(len(rs.predict_proba(test))):
  listtmp = rs.predict_proba(test)[i]
  if listtmp[0] > 0.8:
    negativecase += 1

  elif listtmp[1]> 0.8:
    positiveExtreme += 1
    if listtmp[1] > 0.91:
      listfinal.append(listtmp)
  elif listtmp[0] <= 0.8:
    positivecase += 1

print(negativecase)
print(positivecase)
print(positiveExtreme)

流式输出内容被截断，只能显示最后 5000 行内容。
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.0s finishe

0
2194
135


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished


In [137]:
print(listfinal)

[array([0.01, 0.99]), array([0.04, 0.96]), array([0.02, 0.98]), array([0.03, 0.97]), array([0.03, 0.97]), array([0.04, 0.96]), array([0.04, 0.96]), array([0.02, 0.98]), array([0., 1.]), array([0.04, 0.96]), array([0.03, 0.97]), array([0., 1.]), array([0.01, 0.99]), array([0.03, 0.97]), array([0.01, 0.99]), array([0.03, 0.97])]
